# **Lab 2: Matrix Factorization**
**Edvin von Platen**

# **Abstract**

In this lab we implement, test, and evaluate the following matrix algorithms:

1. Sparse matrix-vector product $b = Ax$,
2. Classical Gram-Schmidt QR Factorization $A=QR$,
3. A Direct Solver using Backsubstitution and QR Factorization $Ax = b \iff QRx = b$,
4. Least Squares problem $Ax = b$. 

Our implementation of the algorithms appear to be sound and they perform quite well when compared to the implementations found in the numpy library.

#**About the code**

In [0]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Edvin von Platen (edvinvp@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
import unittest
import random

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

np.random.seed(seed=1994)
random.seed(1994)

# **Introduction**

We implement and evaluate the following matrix algorithms:

1. Sparse matrix-vector product $b = Ax$,
2. Classical Gram-Schmidt QR Factorization $A=QR$,
3. A Direct Solver using Backsubstitution and QR Factorization $Ax = b \iff QRx = b$,
4. Least Squares problem $Ax = b$.

All implementations and mathematical conccepts presented in this report are based on the lecture notes from the course [DD2363 Methods in Scientific Computing](https://kth.instructure.com/courses/17068). 

In the Methods section we present background and implementation for each algrithm, including unittest implementation. Followed by test results in the Results section and a brief discussion of the results in the Discussion section.


# **Methods**

## **Sparse Matrix-Vector Product**

We implement Algorithm 5.9. sparse matrix-vector product. Input is a  matrix $A \in R^{m\times n}$ represented in compressed row storage (CRS) format, a vector $x \in R^n$, and output is the vector $b = Ax, \ b \in R^m$.

A matrix in the CRS format is represented through three arrays: *val*, *col_idx*, and *row_ptr*. val stores the nonzero matrix values, col_idx the nonzero values column indices, and row_ptr the indices of the of the first nonzero values of each row in val plus the number of vals.

For example, given the matrix $$C = \begin{pmatrix} 0 & 0 & 1 & 0 \\ 2 & 0 & 3 &0 \\ 0 & 0 & 0 & 0  \\ 0 & 5 & 1 & 0 \end{pmatrix} $$
the three arrays are
\begin{align*}
val &= [1,2,3,5,1] \\
col\_idx &= [2,0,2,1,2] \\
row\_idx &= [0,1,3,3,5].
\end{align*}
When there is a zero row the next row index is repeated.

In [0]:
# Using the CRS matrix class from the appendix
# Construct a simple sparse matrix class using the CRS data structure
class spMatrix:

  def __init__(self, m, A = None, val = None, col_idx = None, row_idx = None):
    if A is None:
      self.val = val
      self.col_idx = col_idx
      self.row_idx = row_idx
      # Had to include m to allow for zero rows in matrix-vector multiplication
      self.m = m
    else:
      # Make A sparse
      non_zeros = np.count_nonzero(A)
      v = np.zeros(non_zeros)
      r = []
      non_zero_row, non_zero_col = np.nonzero(A)
      prev_row = -1
      for i in range(len(non_zero_row)):
        v[i] = A[non_zero_row[i], non_zero_col[i]]
        # Check if we have skipped a row
        if (prev_row == non_zero_row[i] - 1):
          # no row was skipped
          prev_row = non_zero_row[i]
          r.append(i)
        elif (prev_row == non_zero_row[i]):
          # Still on same row do nothing
          pass
        else:
          # atleast one row was skipped fill with current idx
          for j in range(prev_row, non_zero_row[i]):
            r.append(i)
          prev_row = non_zero_row[i]

      r.append(len(v))
      self.val = v
      self.col_idx = non_zero_col
      self.row_idx = np.array(r)
      self.m = m

  def print_components(self):
    print("val = " + str(self.val))
    print("col_idx = " + str(self.col_idx))
    print("row_idx = " + str(self.row_idx))

def sparse_matrix_vector_product(A, x):
  k = len(A.row_idx) - 1
  b = np.zeros((A.m,1))
  for i in range(k):
    for j in range(A.row_idx[i], (A.row_idx[i+1])):
      b[i,0] += A.val[j]*x[A.col_idx[j]]
  return b



In [0]:
# Tests for sparse matrix-vector product
class Sparse_matrix_vector_test(unittest.TestCase):
  def test_zero(self):
    A = np.array([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])
    b = np.array([[0],[1],[2],[3]])
    spA = spMatrix(4, A=A)
    b1 = sparse_matrix_vector_product(spA,b)
    b2 = A.dot(b)
    self.assertIs(np.allclose(b1,b2, atol=1e-05), True)

  def test_random_dense(self):
    # run 100 test with different dimensions
    for i in range(10):
      m = random.randint(1,50)
      n = random.randint(1,50)
      A = 1000 * np.random.random_sample((m, n)) - 500 # [-500,500]
      b = 1000 * np.random.random_sample((n,1)) - 500
      spA = spMatrix(m,A=A)
      self.assertIs(np.allclose(sparse_matrix_vector_product(spA,b), A.dot(b), atol=1e-04), True)

  def test_random_sparse(self):
    m = random.randint(50,200)
    n = random.randint(50,200)
    A = np.random.randint(0,10, m * n)
    # make around 10% of the array non-zero
    A[A > 1] = 0
    A = np.reshape(A, (m,n))
    b = np.reshape(np.random.randint(0,10,n), (n,1))
    spA = spMatrix(m, A=A)
    self.assertIs(np.allclose(sparse_matrix_vector_product(spA,b), A.dot(b), atol=1e-04), True)



## **Gram-Schmidt QR Factorization**
Given a non-singular matrix $A\in R^{n\times n}$, its $QR$ factorization is $A=QR$, where $Q$ is an orthogonal matrix and $R$ a upper triangular matrix.

We implement the classical Gram-Schmidt QR iteration. The idea is that since $A$ is non-singular, we can always construct an orthonormal basis $\{q_i\}_{i=1}^j$ such that $<q_1,..,q_j> = <a_{:1},...,a_{:j}>, \ \forall j \leq n$, i.e. $\{q_i\}_{i=1}^j$ and $\{a_{:i}\}_{i=j}^n$ span the same vector space. 

The orthonormal basis $\{q_i\}_{i=1}^n$ is constructed iteratively by equation (5.4),
\begin{align*}
v_j &= a_{:j} - \sum_{i=1}^{j-1} (a_{:j}, q_{j})q_i, \\
q_j &= \frac{v_j}{\Vert v_j \Vert}.
\end{align*}
By rewriting (5.4) as,
\begin{align*}
a_{:1} &= r_{11}q_1 \\
a_{:2} &= r_{12}q_1 + r_{22}q_2 \\
       &\vdots \\
a_{:n} &= r_{1n}q_1 + \dots + r_{nn}q_{n}
\end{align*}
where $r_{ij} = (a_{:j},q_i)$ and $r_{ii} = \Vert v_j \Vert$. Which in matrix form is the $QR$ factorization $A = QR$,

\begin{align*}
\left( \begin{array}{c|c|c|c} & & &  \\ a_{:1} & a_{:2} & \dots & a_{:n} \\ & & & \end{array} \right) = \left( \begin{array}{c|c|c|c} & & &  \\ q_{1} & q_{2} & \dots & q_{n} \\ & & & \end{array} \right)\left( \begin{array}{cccc} r_{11} & r_{12} & \dots & r_{1n} \\  & r_{22} &  &  \\ \vdots & & \ddots & \vdots \\ 0 & \dots & & r_{nn}\end{array} \right)
\end{align*}

In [0]:
def classic_gram_schmidt_qr_iteration(A):
  n = A.shape[0]
  Q = np.zeros((n,n))
  R = np.zeros((n,n))
  for j in range(n):
    a_j = A[:,j]
    s = 0
    for i in range(j):
      q_i = Q[:,i]
      R[i, j] = q_i.dot(a_j)
      s += R[i,j] * q_i
    v_j = a_j - s
    R[j,j] = np.linalg.norm(v_j)
    q_j = v_j / R[j,j]
    Q[:,j] = q_j
  return Q,R

In [0]:
# Tests for classical gram-schmidt iteration
class classic_gram_schmidt_qr_iteration_test(unittest.TestCase):
  def test_random(self):
    # run 100 test with different dimensions
    for i in range(20):
      m = random.randint(1,25)
      A = 1000 * np.random.random_sample((m, m)) - 500 # [-500,500]
      Q1,R1 = classic_gram_schmidt_qr_iteration(A)
      Q2,R2 = np.linalg.qr(A)
      # np.linalg.qr has different signs so we test by checking that QR = A
      A1 = Q1.dot(R1)
      A2 = Q2.dot(R2)
      self.assertIs(np.allclose(A1,A2,atol=1e-05), True)
      
  def test_upper_tri(self):
    # run 20 test with different dimensions
    for i in range(20):
      m = random.randint(1,25)
      A = 1000 * np.random.random_sample((m, m)) - 500 # [-500,500]
      Q1,R1 = classic_gram_schmidt_qr_iteration(A)
      self.assertIs(np.allclose(R1, np.triu(R1), atol=1e-05), True)

  def test_identity(self):
    # run 20 test with different dimensions
    for i in range(20):
      m = random.randint(1,25)
      A = 1000 * np.random.random_sample((m, m)) - 500 # [-500,500]
      Q1,R1 = classic_gram_schmidt_qr_iteration(A)
      I = Q1.dot(np.transpose(Q1))
      self.assertIs(np.allclose(I, np.identity(m), atol=1e-05), True)

## **Direct Solver**

We implement a direct solver of the matrix equation $Ax = b, \ A\in R^{n\times n}, \ b \in R^n, \ x \in R^n$, where $A$ is non-singular. Input is $A$ and $b$ and output $A^{-1} b = x$.

The idea is to use the Gram-Schmidt QR factorization $A = QR$ of the previous section, we then have to solve $QRx = b$. The inverse of an orthogonal matrix is its transpose which gives us $Rx = Q^Tb$, which we can solve using *backward substitution*. 

Given an upper triangular $U \in R^{n\times n}$ matrix we can solve the equation $Ux = b$ using backward substitution. Note that $U_{n,n}x_n = b_n \iff x_n = \frac{b_n}{U_{n,n}}$, we can then substitue $x_n$ into $U_{n-1,n-1}x_{n-1} + U_{n-1, n}x_n = b_{n-1}$, which gives us,
$$
x_{n-1} = \frac{b_{n-1} - U_{n-1,n}\frac{b_n}{U_{n,n}}}{U_{n-1,n-1}}.
$$
And the formula,
$$
x_i = \frac{b_i - \sum_{j=i+1}^n U_{i,j}x_j}{U_{i,i}}
$$

This gives us Algorithm 5.2.



In [0]:
def backward_substitution(U,b):
  n = len(b)
  x = np.zeros((n,1))
  for i in range(n-1, -1, -1):
    s = 0
    for j in range(i+1, n):
      s += U[i,j]*x[j,0]
    x[i,0] = (b[i] - s) / U[i,i]
  return x

def transpose(A):
  m,n = A.shape
  A_T = np.zeros((n,m))
  for i in range(n):
    A_T[i,:] = A[:,i]
  return A_T

def direct_solver(A,b):
  Q,R = classic_gram_schmidt_qr_iteration(A)
  Q_T = transpose(Q)
  Q_Tb = Q_T.dot(b)
  x = backward_substitution(R, Q_Tb)
  return x


In [0]:
# Tests for direct solver
class direct_solver_test(unittest.TestCase):
  def test_random(self):
    # run 20 test with different dimensions
    for i in range(20):
      m = random.randint(1,25)
      A = 1000 * np.random.random_sample((m, m)) - 500 # [-500,500]
      b = 10 * np.random.random_sample((m,1)) - 5
      x1 = direct_solver(A,b)
      x2 = np.linalg.solve(A,b)
      self.assertIs(np.allclose(x1,x2,atol=1e-05), True)

## **Least Squares Problem**

For a system of equations,
\begin{align}
Ax = b,
\end{align}
when $ A\in R^{m\times n}, \ x \in R^n, \ b \in R^m, m > n$ the system is said to be overdecided, i.e. there are more equations then unknowns. Finding a solution to this system is known as a least squares problem. 

An overview of the least squares problem is given in Example 2.14 which we will also give here with some added explanations. Assume that $rank(A^T) = n$, i.e. there are $n$ linearly independent row vectors in $A$. Then, by the fundamental theorem of linear algebra (Theorem 2.2), $n = rank(A^T) + dim(null(A)) \implies null(A) = \emptyset$. The idea is now to project $b$ onto the range of $A$ using an orthogonal projector $P$, i.e. the division of $R^n$ into the two subspaces $range(P)$ and $range(I-P)$ are orthogonal (Section 2.7). This means that $Pb \in range(A)$ with the projection error $b-Pb\in range(A)^\perp$. 

Now, $Pb \in range(A)$ means that there is some $\bar{x}\in R^n$ such that $Pb = A\bar{x}$. This means that the residual of the system of equations is equal to the projection error, $b-Pb = b-A\bar{x}$. Continue by noting that $range(A)^\perp = null(A^T)$, we can see this by observing that we can view the matrix vector multiplication $A^Tx$ as a series of scalar products, 
$$
A^T x = \begin{pmatrix} a_{:1}\cdot x \\ \vdots \\ a_{:m} \cdot x \end{pmatrix}.
$$
Which equals the zero vector only if $x$ orthogonal to all column vectors of $A$. Thus, $A^T(b-A\bar{x}) = 0 \iff A^TA\bar{x} = A^T b$, by Theorem 2.2, this condition is known as the normal equations (2.17). Solving the normal equations is thus the same as finding the best approximation of $b \in R^m$ in $R^n$, i.e. find $\bar{x} \in R^n$ such that
$$
\Vert b - A\bar{x} \Vert \leq \Vert b - Ay \Vert, \ \forall y \in R^n.
$$
Because,
\begin{align}
\Vert Ay - b \Vert^2 &= \Vert A(y-\bar{x}) + (A\bar{x} - b) \Vert^2 \ \ \ \ \ \ 
(\pm A\bar{x}) \\
&= \Vert A(y-\bar{x})\Vert^2 + 2(A(y-\bar{x}))^T(A\bar{x} - b) + \Vert A\bar{x} - b\Vert^2 \\
 & \{ \text{recall that } \Vert (a + d) \Vert^2 = (a,d)\} \\
& \geq 2(y-\bar{x})^TA^T(A\bar{x} - b) + \Vert A\bar{x} - b\Vert^2 = \Vert A\bar{x}- b\Vert^2,
\end{align}
as $A^T(A\bar{x} - b) = 0$. Solving (2.17) for $\bar{x}$ gives us,
$$
\bar{x} = (A^T A)^{-1}A^Tb.
$$
We now give an implementation for solving a given least squares problem using the previously implemented classical Gram-Schmidt iteration.

In [0]:
def least_squares(A,b):
  A_T = transpose(A)
  A_TA = A_T.dot(A)
  A_Tb = A_T.dot(b)
  Q, R = classic_gram_schmidt_qr_iteration(A_TA)
  Q_T = transpose(Q)
  Q_TA_Tb = Q_T.dot(A_Tb)
  x = backward_substitution(R, Q_TA_Tb)
  return x

In [0]:
# Tests for Least Squares
class least_squares_test(unittest.TestCase):
  def test_random(self):
    # run 20 test with different dimensions
    for i in range(20):
      m = random.randint(5,10)
      n = random.randint(1, m)
      A = 1000 * np.random.random_sample((m, n)) - 500 # [-500,500]
      b = np.random.random_sample((m,1))
      x1 = least_squares(A,b)
      x2 = np.linalg.solve(np.transpose(A).dot(A),np.transpose(A).dot(b))
      self.assertIs(np.allclose(x1,x2,atol=1e-05), True)

# **Results**

In this section we will present the results of the tests for the implemented algorithms.

First we evaulate all unittests in the previous section which cover, I was inspired to use unittests after reviewing Kristoffer Almroths lab 1 report.

- **sparse_matrix_vector_multiplication(A,b)**: Test performace against np.dot() for both sparse- and dense matrices. Test behaviour when multiplying with all zero matrix.
- **classical_gram_schmidt_qr_factorization(A)**: Test against np.lingalg.qr() for randomized matrices. Test that $R$ is upper triangular, test that $Q$ is orthogonal.
- **direct_solver(A,b)**: Test against np.linalg.solve() for randomized matrices.
- **least_squares(A,b)**: Test against np.linalg.solve() for the normal equastions calculated using np functions for randomized matrices.

In [29]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

........
----------------------------------------------------------------------
Ran 8 tests in 0.173s

OK


All tests pass.

#### **Gram-Schmidt QR Iteration**

We continue by evaluating the Frobenius norms $\Vert Q^TQ - I\Vert_F, \Vert Q R-A\Vert_F$ for the classical Gram-Schmidt QR iteration implementation. The Frobenious norm is defined for a $m\times n$ matrix $A$ as,
$$
\Vert A \Vert_F = trace(A^TA) = \left( \sum_{i=1}^m\sum_{j=1}^n |a_{ij}|^2\right)^{1/2} \ \ \ (section \ 2.5).
$$
The Frobenious norm is the default norm in the np.linalg.norm() function. We evaluate the two Frobenious norms for 20 randomized matrices. We also compare the result with the np.linalg.qr() function.


In [30]:
for i in range(10):
  m = random.randint(1,100)
  A = 1000 * np.random.random_sample((m, m)) - 500 # [-500,500]
  Q,R = classic_gram_schmidt_qr_iteration(A)
  Q2,R2 = np.linalg.qr(A)
  print("A dim: " + str(m) + " times " + str(m))
  print("REPORT IMPLEMENTATION")
  print("||Q^TQ-I||_F = " + str(np.linalg.norm(np.transpose(Q).dot(Q) - np.identity(m))))
  print("||QR - A||_F =" + str(np.linalg.norm(Q.dot(R) - A))) 
  print("USING np.linalg.qr()")
  print("||Q^TQ-I||_F = " + str(np.linalg.norm(np.transpose(Q2).dot(Q2) - np.identity(m))))
  print("||QR - A||_F =" + str(np.linalg.norm(Q2.dot(R2) - A))) 

A dim: 14 times 14
REPORT IMPLEMENTATION
||Q^TQ-I||_F = 1.7268732252062096e-14
||QR - A||_F =3.2241029558033507e-13
USING np.linalg.qr()
||Q^TQ-I||_F = 2.148927943038132e-15
||QR - A||_F =1.8364837177524834e-12
A dim: 60 times 60
REPORT IMPLEMENTATION
||Q^TQ-I||_F = 5.754687946844909e-13
||QR - A||_F =2.2608797146494282e-12
USING np.linalg.qr()
||Q^TQ-I||_F = 5.4463290832238415e-15
||QR - A||_F =9.225592823923896e-12
A dim: 61 times 61
REPORT IMPLEMENTATION
||Q^TQ-I||_F = 9.169919903451057e-14
||QR - A||_F =2.4243252002880867e-12
USING np.linalg.qr()
||Q^TQ-I||_F = 5.12419973155947e-15
||QR - A||_F =9.327007245872977e-12
A dim: 50 times 50
REPORT IMPLEMENTATION
||Q^TQ-I||_F = 6.064871935216519e-14
||QR - A||_F =1.8980225389607257e-12
USING np.linalg.qr()
||Q^TQ-I||_F = 4.46409627713528e-15
||QR - A||_F =7.016707848843544e-12
A dim: 87 times 87
REPORT IMPLEMENTATION
||Q^TQ-I||_F = 3.472236634550275e-13
||QR - A||_F =3.753021350021316e-12
USING np.linalg.qr()
||Q^TQ-I||_F = 7.24193709262

### **Direct Solver**

To test the implemented direct solver we inspect the residual $\Vert Ax - b\Vert$ and $\Vert x-y \Vert$ where $y$ is a manufactured solution with $b=Ay$.

We start by inspecting the residual for the solution to 10 randomized systems of equations, and comparing to the np.linalg.solve() function.

In [31]:
for i in range(10):
  m = random.randint(1,100)
  A = 1000 * np.random.random_sample((m, m)) - 500 # [-500,500]
  b = 1000 * np.random.random_sample((m,1)) - 500
  x = direct_solver(A,b)
  x2 = np.linalg.solve(A,b)
  print("A dim: " + str(m) + " times " + str(m))
  print("REPORT IMPLEMENTATIOn")
  print("||Ax-b|| = " + str(np.linalg.norm(A.dot(x)- b)))
  print("USING np.linalg.solve()")
  print("||Ax-b|| = " + str(np.linalg.norm(A.dot(x2)- b)))

A dim: 9 times 9
REPORT IMPLEMENTATIOn
||Ax-b|| = 2.412875915867585e-12
USING np.linalg.solve()
||Ax-b|| = 1.4224526086681128e-12
A dim: 63 times 63
REPORT IMPLEMENTATIOn
||Ax-b|| = 2.789310328345715e-10
USING np.linalg.solve()
||Ax-b|| = 6.553890896393737e-11
A dim: 19 times 19
REPORT IMPLEMENTATIOn
||Ax-b|| = 1.6177920623201438e-12
USING np.linalg.solve()
||Ax-b|| = 1.1058930600906773e-12
A dim: 39 times 39
REPORT IMPLEMENTATIOn
||Ax-b|| = 5.337503199215519e-11
USING np.linalg.solve()
||Ax-b|| = 3.342064326417041e-11
A dim: 9 times 9
REPORT IMPLEMENTATIOn
||Ax-b|| = 3.152920801928444e-13
USING np.linalg.solve()
||Ax-b|| = 2.773848179905249e-13
A dim: 89 times 89
REPORT IMPLEMENTATIOn
||Ax-b|| = 3.2928507606123625e-11
USING np.linalg.solve()
||Ax-b|| = 2.1044223507251803e-11
A dim: 39 times 39
REPORT IMPLEMENTATIOn
||Ax-b|| = 1.0080175068195804e-11
USING np.linalg.solve()
||Ax-b|| = 1.3875187745221335e-11
A dim: 91 times 91
REPORT IMPLEMENTATIOn
||Ax-b|| = 2.5224271835585427e-10
USING

To test a manufactured solution we use the system,
$$
\begin{pmatrix} 1 & 2 & 3 \\ 1 & 0 & 3 \\ 0 & 4 & 1 \end{pmatrix} \begin{pmatrix} x \\ y \\ z \end{pmatrix} = \begin{pmatrix} 1 \\ 2 \\ 3 \end{pmatrix}
$$
which has the solution,
$$
\begin{pmatrix} x \\ y \\ z \end{pmatrix} = \begin{pmatrix} -13 \\ -\frac{1}{2} \\ 5 \end{pmatrix}$$.

In [32]:
A = np.array([[1,2,3],[1,0,3],[0,4,1]])
b = np.array([[1],[2],[3]])
y = np.array([[-13],[-0.5],[5]])
x = direct_solver(A,b)
print(x)
print(np.linalg.norm(x - y))

[[-13. ]
 [ -0.5]
 [  5. ]]
2.094764613337708e-15


### **Least Squarres**

To test our least squares implementation we will again inspect the residual $\Vert Ax - b \Vert$ for 10 randomized least square problems, and compare against calculating the normal equations and solving using np.linalg.

In [33]:
for i in range(20):
  m = random.randint(7,25)
  n = random.randint(5, m-1)
  A = 1000 * np.random.random_sample((m, n)) - 500 # [-500,500]
  b = 1000 * np.random.random_sample((m,1)) - 500
  x = least_squares(A,b)
  x2 = np.linalg.solve(np.transpose(A).dot(A),np.transpose(A).dot(b))
  print("A dim: " + str(m) + " times " + str(n))
  print(" A dim diff = " + str(m-n))
  print("REPORT IMPLEMENTATION")
  print("||Ax - b|| = "  + str(np.linalg.norm(A.dot(x) - b)))
  print("USING np.linalg")
  print("||Ax - b|| = "  + str(np.linalg.norm(A.dot(x2) - b)))
  print("DIFF: " + str(np.abs(np.linalg.norm(A.dot(x) - b)- np.linalg.norm(A.dot(x2) - b))))


A dim: 15 times 5
 A dim diff = 10
REPORT IMPLEMENTATION
||Ax - b|| = 908.4097805155958
USING np.linalg
||Ax - b|| = 908.4097805155959
DIFF: 1.1368683772161603e-13
A dim: 7 times 5
 A dim diff = 2
REPORT IMPLEMENTATION
||Ax - b|| = 25.550174719595134
USING np.linalg
||Ax - b|| = 25.550174719595176
DIFF: 4.263256414560601e-14
A dim: 16 times 14
 A dim diff = 2
REPORT IMPLEMENTATION
||Ax - b|| = 216.99672360820995
USING np.linalg
||Ax - b|| = 216.99672360821
DIFF: 5.684341886080802e-14
A dim: 19 times 18
 A dim diff = 1
REPORT IMPLEMENTATION
||Ax - b|| = 172.2324629895131
USING np.linalg
||Ax - b|| = 172.23246298951304
DIFF: 5.684341886080802e-14
A dim: 14 times 10
 A dim diff = 4
REPORT IMPLEMENTATION
||Ax - b|| = 576.1907877237278
USING np.linalg
||Ax - b|| = 576.1907877237279
DIFF: 1.1368683772161603e-13
A dim: 9 times 7
 A dim diff = 2
REPORT IMPLEMENTATION
||Ax - b|| = 386.13386804445196
USING np.linalg
||Ax - b|| = 386.133868044452
DIFF: 5.684341886080802e-14
A dim: 22 times 20
 A 

# **Discussion**

All of the implemented algorithms appear to be working properly.

**Sparse Matrix-Vector Product**: The implementation passed all tests against the np.dot() function. 

**Gram-Schmidt QR Factorization**: The residual was quite small for all tests, and R appears to always be upper-triangular. It was quite surprising that the residuals was so small even though the values of the matrices were in the range of $[-500,500)$, as the classical Gram-Schmidt QR factorization has some problems with numerical stability (section 5.3). However, the test cases did probably not cover such instances. The np.linalg.qr() function appear to have a slight edge compared to our implementation.

**Direct Solver**: The residual was quite small for all tests, so the algorithm appear to perform quite well. The residual for our implementation is also very close the the residual using np.linalg.solve().

**Least Squares**: Here we got a much larger residual then for the direct solver, which is expected as the system is overdecided and we are projecting $b$ onto $A$. It appears that the residual is increasing as the difference in dimension between $A$ and $b$ increase, this is likely due to the projection, but could also be due to numerical instability as the system size increases. Surprisingly our implementation performs almost identically to using numpy for all calculations.